In [14]:
from pickle import load
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score, precision_score, recall_score
from sklearn import svm
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipe

In [15]:
df = load(open('df_clean.pkl', 'rb'))

In [16]:
df.head()

,text,company,is_ther,target,product
0,wesley83 i have a 3g iphone after 3 hrs tweeti...,apple,Negative emotion,Negative emotion,iphone
1,jessedee know about fludapp awesome ipadiphon...,apple,Positive emotion,Positive emotion,ipad or iphone app
2,swonderlin can not wait for ipad 2 also they s...,apple,Positive emotion,Positive emotion,ipad
3,sxsw i hope this years festival isnt as crashy...,apple,Negative emotion,Negative emotion,ipad or iphone app
4,sxtxstate great stuff on fri sxsw marissa maye...,google,Positive emotion,Positive emotion,google


In [17]:
X = df['text']
y = df.target

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [19]:
sw = stopwords.words('english')

In [20]:
# oversample = SMOTE()
# X_train, y_train = oversample.fit_resample(X_train, y_train)

In [21]:
micro_recall = make_scorer(recall_score, average = 'micro')

In [22]:
# dt_pipe = imbpipe(steps=[
#     ('ss', StandardScaler()),
#     ('sm', SMOTE(random_state=42)),
#     ('dtc', DecisionTreeClassifier(random_state=42, 
#                                    max_depth= 6, 
#                                    class_weight = 'balanced'))])

In [23]:
pipe = imbpipe(steps = [
    ('tfid', TfidfVectorizer(max_features=300, stop_words= sw)), 
    ('sm', SMOTE()), 
    ('svc', svm.SVC())])


In [24]:
cv = cross_validate(pipe, X_train, y_train, scoring = micro_recall)
cv

{'fit_time': array([4.6535604 , 5.02594495, 4.6375134 , 4.96406054, 4.71123528]),
 'score_time': array([0.53954649, 0.63450885, 0.54296136, 0.63841295, 0.6712122 ]),
 'test_score': array([0.59983566, 0.6409203 , 0.63188168, 0.63516845, 0.61955629])}

In [25]:
y_hat_train = cross_val_predict(pipe, X_train, y_train)

In [26]:
confusion_matrix(y_train, y_hat_train)

array([[  88,  243,   94],
       [ 144, 2659,  639],
       [  68, 1104, 1046]], dtype=int64)